## Create the embeddings and save them to file
Savce the embeddings so that they can be reused later

### Import the necessary libraries

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import json

### Set the paths and files

In [3]:
use_path = "dataset/"
baseline_file = "baseline_questions.csv"
quora_file = "quora_questions.csv"

### Download the USE module

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
embed = hub.Module(module_url)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/v

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

### Create an interactive session to use later when running the model

In [5]:
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

### Create a dataframe of quora question pairs and answer labels 

In [6]:
def get_quora_qs():
    quora_path = use_path + quora_file
    quora_qs = pd.read_csv(quora_path)
    return(quora_qs)

### Generate embeddings for each of the questions

In [7]:
sts_input1 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)

def get_embeds(session, text):
    """Returns the similarity scores"""
    embed = session.run(
        [sts_encode1],
        feed_dict={
            sts_input1: text
        })
    return(embed)

### Create a dataframe of queries

In [8]:
quora_df = get_quora_qs()
quora_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
1,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
2,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
3,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
4,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [9]:
# Combine both question columns into df with one column
q_1_2 = pd.concat([quora_df['question1'], quora_df['question2']])
questions = [e for e in q_1_2]

In [10]:
%%time
embeddings = get_embeds(session, questions)

CPU times: user 3min 10s, sys: 12.4 s, total: 3min 22s
Wall time: 27 s


In [11]:
res = (embeddings[0]).tolist()

In [15]:
q_df = pd.DataFrame(questions, columns=['questions'])
q_df['embeddings'] = res

In [17]:
q_df.head()

,questions,embeddings
0,Astrology: I am a Capricorn Sun Cap moon and c...,"[-0.043053239583969116, 0.05912359803915024, -..."
1,How can I be a good geologist?,"[0.04468201845884323, 0.0032161467242985964, 0..."
2,How do I read and find my YouTube comments?,"[0.031296346336603165, -0.002411748981103301, ..."
3,What can make Physics easy to learn?,"[0.0734066367149353, 0.055123668164014816, 0.0..."
4,What was your first sexual experience like?,"[0.05466366186738014, -0.003594229696318507, -..."


In [21]:
q_df.to_pickle("/home/ec2-user/sentence_embeddings/embeddings.pkl") 